In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from os import listdir, environ
from os.path import isfile, join
from pathlib import Path
import scipy.stats as stats
import itertools
from tqdm import tqdm
from scipy.stats import rankdata

In [2]:
path_election = '../../data/ronaldo/data/TSE/RS/data/aggregated_by_weighting_area.csv'
path_matrix = '../../data/ronaldo/data/IBGE/adjacency_matrices/weighting_areas/queen.csv'

In [3]:
data_election = pd.read_csv(path_election)
adjacency_matrix = pd.read_csv(path_matrix)
adjacency_matrix.set_index('Unnamed: 0', inplace=True)
adjacency_matrix.sort_index(inplace=True)

#data_election = data_election[data_election['NM_MUNICIPIO']=='PORTO ALEGRE']
data_election = data_election[data_election['NM_MUNICIPIO']=='RIO GRANDE']
#data_election = data_election[data_election['NM_MUNICIPIO']=='CAXIAS DO SUL']
data_election.head()

,Cod_ap,SG_ UF,NM_MUNICIPIO,CD_MUNICIPIO,COD_LOCALIDADE_IBGE,NR_ZONA,NR_LOCAL_VOTACAO,local_unico,NR_SECAO,rural,...,lev_dist,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_ELEITORES_BIOMETRIA_NH,Branco,Nulo,JAIR BOLSONARO,FERNANDO HADDAD,precision
570,4.315602e+12,RS,RIO GRANDE,88153,4315602.0,"[163, 163, 163, 163, 163, 163, 163, 163, 163]","[1120, 1147, 1155, 1163, 1171, 1201, 1210, 123...","SALÃO COMUNITÁRIO NOSSA SENHORA DE FÁTIMA, ILH...","['[49]', '[38]', '[200, 47]', '[150, 206, 245,...",False,...,1.0,11500,8938,2562,1422.0,347,766,3695,4130,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
571,4.315602e+12,RS,RIO GRANDE,88153,4315602.0,[163],[1112],"ESCOLA MUNICIPAL FRANCA PINTO, RUA MINAS GERAI...","['[120, 13, 14, 162, 208, 251, 287, 288, 336, ...",False,...,1.0,4627,3636,991,451.0,105,267,1432,1832,['TSE']
572,4.315602e+12,RS,RIO GRANDE,88153,4315602.0,"[163, 163, 163]","[1074, 1260, 1350]","ESCOLA ESTADUAL ROBERTO BASTOS TELLECHEA, AVEN...","['[105, 122, 142, 159, 180, 195, 210, 221, 230...",False,...,1.0,14296,11494,2802,1280.0,336,905,4491,5762,"['TSE', 'TSE', 'TSE']"
573,4.315602e+12,RS,RIO GRANDE,88153,4315602.0,"[37, 37, 37, 37, 37]","[1392, 1406, 1449, 1457, 1465]","ESCOLA ESTADUAL DE ENSINO MÉDIO SILVA GAMA, RU...","['[123, 139, 173, 196, 216, 232, 247, 25, 258,...",False,...,1.0,14792,11318,3474,1494.0,293,627,5109,5289,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE']"
574,4.315602e+12,RS,RIO GRANDE,88153,4315602.0,"[163, 37, 37]","[1198, 1414, 1422]","ESCOLA AURORA FERREIRA CADAVAL, VILA DO TAIM, ...","['[270, 48]', '[325, 335, 353, 373, 389, 398, ...",False,...,1.0,5534,4265,1269,538.0,123,289,1705,2148,"['TSE', 'TSE', 'TSE']"


In [4]:
data_election.set_index('Cod_ap', inplace=True)
data_election.sort_index(inplace=True)

In [5]:
votes = data_election[['JAIR BOLSONARO', 'FERNANDO HADDAD']].copy()
votes['FERNANDO HADDAD'] = votes['FERNANDO HADDAD'] / (votes['FERNANDO HADDAD'] + votes['JAIR BOLSONARO'])
votes['JAIR BOLSONARO'] = 1 - votes['FERNANDO HADDAD']
votes.head()

,JAIR BOLSONARO,FERNANDO HADDAD
Cod_ap,,
4.315602e+12,0.472204,0.527796
4.315602e+12,0.438725,0.561275
4.315602e+12,0.438018,0.561982
4.315602e+12,0.491344,0.508656
4.315602e+12,0.442512,0.557488


In [6]:
feature = np.array(votes['JAIR BOLSONARO'].values).reshape(-1,1)
votes_sm = distance.pdist(feature,'cityblock')
votes_sm = pd.DataFrame(distance.squareform(votes_sm), index=votes.index, columns= votes.index)
votes_sm.head()

Cod_ap,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12
Cod_ap,,,,,,,,,,,
4.315602e+12,0.000000,0.033479,0.034186,0.019140,0.029692,0.029741,0.031894,0.037571,0.007736,0.012836,0.056612
4.315602e+12,0.033479,0.000000,0.000707,0.052619,0.003787,0.003737,0.065373,0.071050,0.041215,0.020643,0.023133
4.315602e+12,0.034186,0.000707,0.000000,0.053326,0.004494,0.004445,0.066080,0.071758,0.041923,0.021350,0.022425
4.315602e+12,0.019140,0.052619,0.053326,0.000000,0.048832,0.048882,0.012754,0.018431,0.011404,0.031976,0.075752
4.315602e+12,0.029692,0.003787,0.004494,0.048832,0.000000,0.000049,0.061586,0.067263,0.037428,0.016856,0.026920


In [7]:
wa_remove = [x for x in adjacency_matrix.index if x not in votes_sm.index]
for wa in wa_remove:
    adjacency_matrix.drop(wa, axis = 0, inplace=True)
    adjacency_matrix.drop(str(wa), axis = 1, inplace=True)
adjacency_matrix.shape

(11, 11)

In [8]:
votes_sm = pd.DataFrame(votes_sm.values*adjacency_matrix.values, columns=votes_sm.index, index=votes_sm.index)
votes_sm.head()

Cod_ap,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12,4.315602e+12
Cod_ap,,,,,,,,,,,
4.315602e+12,0.0,0.0,0.0,0.000000,0.029692,0.029741,0.0,0.000000,0.000000,0.000000,0.00000
4.315602e+12,0.0,0.0,0.0,0.052619,0.003787,0.000000,0.0,0.000000,0.000000,0.000000,0.00000
4.315602e+12,0.0,0.0,0.0,0.000000,0.000000,0.004445,0.0,0.071758,0.000000,0.021350,0.00000
4.315602e+12,0.0,0.0,0.0,0.000000,0.000000,0.048882,0.0,0.000000,0.011404,0.000000,0.00000
4.315602e+12,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.067263,0.037428,0.016856,0.02692


In [9]:
votes_strangness = votes_sm.sum(axis=0)
#votes_strangness.sort_values(ascending=False, inplace=True)
rank_votes = rankdata(-votes_strangness, method='ordinal')
rank_votes = pd.DataFrame(rank_votes, index=votes_strangness.index)

In [10]:
path_census = '../../data/ronaldo/data/IBGE/census_2010/RS/aggregated_by_weighting_area/joined/'
filenames = [filename for filename in listdir(path_census) if isfile(join(path_census, filename))]
filenames

['Alfabetizacao.csv',
 'Cor_e_Raca.csv',
 'Domicilio.csv',
 'Entorno.csv',
 'Parentesco.csv',
 'Pessoa.csv',
 'Renda.csv',
 'Responsavel.csv']

In [11]:
census_rank = dict()
for file in filenames:
    census_data = pd.read_csv(path_census+file)
    census_data.set_index('Cod_ap', inplace=True)
    census_data.sort_index(inplace=True)
    census_data.drop('CD_GEOCODM', axis=1, inplace=True)
    census_data.drop('NM_MUNICIP', axis=1, inplace=True)
    census_data.drop(wa_remove, axis=0, inplace=True)
    census_data=(census_data-census_data.min())/(census_data.max()-census_data.min())
    census_sm = distance.pdist(census_data,'cityblock')
    census_sm = pd.DataFrame(distance.squareform(census_sm), index=census_data.index, columns= census_data.index)
    census_sm = pd.DataFrame(census_sm.values*adjacency_matrix.values, columns=census_sm.index, index=census_sm.index)
    census_strangness = census_sm.sum(axis=0)
    sorted_rank = rankdata(-census_strangness, method='ordinal')
    sorted_rank = pd.DataFrame(sorted_rank, index=census_strangness.index)
    file = file.split('.')[0]
    census_rank[file] = sorted_rank

In [12]:
def spearman_footrule(a,b):
    w = np.arange(1,len(a)+1)/len(a)
    w = w[::-1]
    a = a.sort_values(by=0)
    w = pd.DataFrame(w,index=a.index)
    diff = abs(a.sub(b))
    d = diff.multiply(w)
    return d[0].sum()

def max_dist_spearman_footrule(a):
    w = np.arange(1,len(a)+1)/len(a)
    w = w[::-1]
    a = a.sort_values(by=0)
    w = pd.DataFrame(w,index=a.index)
    inv_rank = np.arange(len(a),0,-1)
    inv_rank = pd.DataFrame(inv_rank,index=a.index)
    d = abs(a-inv_rank).multiply(w)
    return d[0].sum()

def spearman_footrule_normalized(a,b):
    num = spearman_footrule(a,b)
    denom = max_dist_spearman_footrule(a)
    return float(num)/denom

In [13]:
kendal_rank_cor = dict()
spearman_rank_cor = []
for file in census_rank:
    tau, p_value = stats.weightedtau(rank_votes, census_rank[file])
    sp_dist = spearman_footrule_normalized(rank_votes, census_rank[file])
    kendal_rank_cor[file] = [tau]
    spearman_rank_cor.append(sp_dist)

In [14]:
rank_similarity = pd.DataFrame.from_dict(kendal_rank_cor,orient='index')
rank_similarity['sp_dist'] = spearman_rank_cor
rank_similarity[['sp_dist']].sort_values(by='sp_dist')

,sp_dist
Cor_e_Raca,0.350000
Alfabetizacao,0.397222
Renda,0.497222
Domicilio,0.913889
Entorno,0.913889
Parentesco,0.913889
Pessoa,0.913889
Responsavel,0.913889


In [15]:
census_rank = dict()
kendal_rank_cor = dict()
spearman_rank_dist = dict()
for file in filenames:
    print(file)
    min_tau = -100000000
    max_sp = 1000000000000000000 
    census_data = pd.read_csv(path_census+file)
    census_data.set_index('Cod_ap', inplace=True)
    census_data.sort_index(inplace=True)
    census_data.drop('CD_GEOCODM', axis=1, inplace=True)
    census_data.drop('NM_MUNICIP', axis=1, inplace=True)
    census_data.drop(wa_remove, axis=0, inplace=True)
    file = file.split('.')[0]
    census_data=(census_data-census_data.min())/(census_data.max()-census_data.min())
    for col in tqdm(census_data.columns.values):
        feature = np.array(census_data[col].values).reshape(-1,1)
        census_sm = distance.pdist(feature,'cityblock')
        census_sm = pd.DataFrame(distance.squareform(census_sm), index=census_data.index, columns= census_data.index)
        census_sm = pd.DataFrame(census_sm.values*adjacency_matrix.values, columns=census_sm.index, index=census_sm.index)
        census_strangness = census_sm.sum(axis=0)
        sorted_rank = rankdata(-census_strangness, method='ordinal')
        sorted_rank = pd.DataFrame(sorted_rank, index=votes_strangness.index)
        tau, p_value = stats.weightedtau(rank_votes, sorted_rank)
        sp_dist = spearman_footrule_normalized(rank_votes,sorted_rank)
        
        if tau > min_tau:
            min_tau = tau
            kendal_rank_cor[file] = [col,tau]
        if sp_dist < max_sp:
            max_sp = sp_dist
            spearman_rank_dist[file] = [col,sp_dist]
        

  0%|                                                              | 0/255 [00:00<?, ?it/s]

Alfabetizacao.csv


100%|███████████████████████████████████████████████████| 255/255 [00:01<00:00, 158.49it/s]


Cor_e_Raca.csv


100%|███████████████████████████████████████████████████| 416/416 [00:02<00:00, 159.87it/s]


...

  0%|                                                              | 0/278 [00:00<?, ?it/s]

Renda.csv


100%|███████████████████████████████████████████████████| 278/278 [00:01<00:00, 160.05it/s]


Responsavel.csv


100%|███████████████████████████████████████████████████| 324/324 [00:02<00:00, 160.71it/s]


In [16]:
kendal_rank_cor = pd.DataFrame.from_dict(kendal_rank_cor,orient='index')
spearman_rank_dist = pd.DataFrame.from_dict(spearman_rank_dist,orient='index')
spearman_rank_dist.sort_values(by=1)

,0,1
Parentesco,01_V159,0.136111
Pessoa,02_V085,0.158333
Cor_e_Raca,01_V155,0.177778
Responsavel,02_V137,0.191667
Alfabetizacao,02_V111,0.202778
Renda,03_V035,0.233333
Entorno,02_V278,0.247222
Domicilio,02_V131,0.272222


In [17]:
kendal_rank_cor

,0,1
Alfabetizacao,01_V029,0.841046
Cor_e_Raca,01_V206,0.832268
Domicilio,01_V125,0.785436
Entorno,01_V195,0.786882
Parentesco,01_V159,0.880264
Pessoa,03_V066,0.878065
Renda,02_V049,0.721595
Responsavel,02_V134,0.815101
